In [1]:
# import numpy as np
# from sklearn.preprocessing import Imputer

# ratings = data['review_scores_rating']
# imp = Imputer(missing_values=np.nan, strategy='mean', axis=1)
# ratings = np.array(ratings).reshape(1, -1)
# imp.fit(ratings)
# ratings = imp.transform(ratings)
# ratings.shape
# sample_idxs = np.random.randint(ratings.shape[1], size=1000)
# sample_idxs
# np.take(ratings, sample_idxs)

In [2]:
# import math
# import matplotlib.pyplot as plt
# plt.hist(np.take(ratings, sample_idxs))
# len(data.columns)
# len(data.sample(1).values[0])
# for k, v in zip(data.columns, data.sample(1).values[0]): print k + ': ' + str(v)

In [3]:
import pandas as pd

In [4]:
all_data = pd.read_csv('./data/review_features.csv', low_memory=False)

In [ ]:
data = all_data.sample(20000, random_state=1234)
cols_to_drop = ['access', 'amenities_', 'description', \
               'house_rules', 'id', 'host_verifications', 'last_review', 'last_scraped', \
               'market', 'neighborhood_overview', 'neighbourhood', 'neighbourhood_cleansed', \
               'neighbourhood_group_cleansed', 'notes', 'street', 'summary', 'transit', \
               'Unnamed: 0', 'first_review', 'calendar_last_scraped', 'calendar_updated', 
               'review_scores_accuracy', 'review_scores_checkin', 'review_scores_cleanliness', \
                'review_scores_communication', 'review_scores_location']
data.loc[:, cols_to_drop]
data = data.drop(cols_to_drop, axis=1)

data = data.replace('f', 0)
data = data.replace('t', 1)
data = data.replace(r'[\$,]','', regex=True)
# data['review_scores_rating']


# Get a list of non-numeric columns so we can drop them
from collections import defaultdict

non_numeric_columns = defaultdict(int)

for d in data.values:
    for k, v in zip(data.columns, d):
        if 'Mission' == v:
            print k, v
        
        try: 
            float(v)
        except Exception as e:
            non_numeric_columns[k] += 1

print non_numeric_columns.keys()
data = data.drop(non_numeric_columns.keys(), axis=1)

In [ ]:
review_columns = defaultdict(int)
for c in data.columns:
    if 'review' in c:
        review_columns[c] += 1
review_columns.keys()

In [ ]:
dependent_variable = 'review_scores_rating'

In [ ]:
# Drop missing values from depedent variable
data = data.dropna(subset=[dependent_variable])

In [ ]:
y = data[dependent_variable]
X = data.drop(review_columns.keys(), axis=1)

In [ ]:
for c in X.columns:
    if 'review' in c:
        print c

In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.33, 
                                                    random_state=1234)

In [ ]:
import numpy as np
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values=np.nan, strategy='median', axis=1)
imp.fit(X_train)
X_train = imp.transform(X_train)

imp = Imputer(missing_values=np.nan, strategy='median', axis=1)
imp.fit(X_test)
X_test = imp.transform(X_test)

In [ ]:
y_train = np.array(y_train).reshape(1, -1)
imp = Imputer(missing_values=np.nan, strategy='median', axis=1)
imp.fit(y_train)
y_train = imp.transform(y_train)
# y_train = y_train.reshape(-1, 1)
y_train = y_train.ravel()

y_test = np.array(y_test).reshape(1, -1)
imp = Imputer(missing_values=np.nan, strategy='median', axis=1)
imp.fit(y_test)
y_test = imp.transform(y_test)
# y_test = y_test.reshape(-1, 1)
y_test = y_test.ravel()

In [ ]:
print X_train.shape
print y_train.ravel().shape

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, make_scorer, roc_auc_score

# Create linear regression object
regr = linear_model.Lasso()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = map(lambda x: round(x, 0), regr.predict(X_test))

# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, y_pred))


In [ ]:
# MSE without dropping nan from predictor
# Mean squared error: 47.72
# Variance score: 0.08


In [ ]:
for i, j in zip(y_test[:20], y_pred[:20]): print i, j

In [ ]:
# With imputing by mean
# Mean squared error: 18099081.99
# Variance score: -371456.56

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, neural_network
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, make_scorer, roc_auc_score

# Create linear regression object
nn = neural_network.MLPRegressor(alpha=0.7, warm_start=False, activation='logistic', \
                                learning_rate='invscaling', solver='sgd', max_iter=1000)

# Train the model using the training sets
nn.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = map(lambda x: round(x, 0), nn.predict(X_test))

# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, y_pred))


In [ ]:
for i, j in zip(y_test[:20], y_pred[:20]): print i, j